# Quantum Machine Learning in Quantum Syncronization

In [79]:
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.algorithms.optimizers import COBYLA
from qiskit_machine_learning.datasets import ad_hoc_data
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.stats import pearsonr

collision_model = pd.read_csv("syncro_4000_teste.csv", header=None)

In [113]:
%%time
def preparing_data(initial_data, meas, data_amount):
    pc = list()
    data = list()
    for i in range(0, data_amount):
        l1 = list(initial_data.values[i, 3:(3+meas)])
        l2 = list(initial_data.values[i, 4003:(4003+meas)])
        data.append(l1+l2)
        pc.append(pearsonr(initial_data.values[i, 3903:4003], initial_data.values[i,7903:8003])[0])
    
    return pd.DataFrame(data), pc


# Load the training data
data, pc = preparing_data(collision_model, 5, len(collision_model))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, pc, test_size=0.25, random_state=42)

# Define the feature map
feature_map = ZZFeatureMap(feature_dimension=10, reps=3)

# Define the optimizer
optimizer = COBYLA(maxiter=100)

# Define the quantum instance to be used for running the circuits
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))

# Define the quantum machine learning classifier
vqc = VQC(optimizer=optimizer, feature_map=feature_map, ansatz=None, quantum_instance=quantum_instance)

# Fit the model to the training data
vqc.fit(pd.DataFrame(X_train), np.array(y_train))
print(vqc.weights)

# Evaluate the model on the test data
y_pred = vqc.predict(X_test)

# Print the R^2 score of the model
from sklearn.metrics import r2_score
print(f"R^2 score: {r2_score(y_test, y_pred)}")


[ 1.06065065  1.61634178  0.00951539  1.9512119   0.64558602  0.18185422
 -0.08789541  0.19496747  1.56732568  0.37576659  2.131399    0.39437762
  0.31417204  0.01031596  0.67373844  0.11995271  0.57659705  0.53195332
  0.87478847  0.75230003  0.04686531  1.5331638   0.1768588   0.32653889
  0.04334742  0.16049605  0.11843264  0.2764505   0.41913649  0.24709032
  0.4780718   0.33491155  0.16060819  0.26749647  0.2664785   0.13441459
 -0.17701475  0.26651995  0.2407078   0.03790613]
R^2 score: -1.9491699798779045
CPU times: user 6h 3min 16s, sys: 1h 1min 16s, total: 7h 4min 33s
Wall time: 7h 46min 22s


In [114]:
y_pred

array([-0.99772351, -0.98942065, -0.99772351, ..., -0.98942065,
       -0.98451505, -0.99772351])

In [112]:
y_test

[-0.8064939379871592,
 -0.9864919606885252,
 0.7908907139685943,
 0.7325536921405832,
 0.11307369305381376,
 -0.9982745710897464,
 -0.9976138689393521,
 -0.9854992515341232,
 -0.995240306213395,
 -0.9977638987600987,
 -0.6464419404282415,
 -0.9985594275715723,
 -0.9965380548291922,
 -0.6199022077081224,
 -0.9975360815370737,
 -0.9974693515556147,
 -0.523690443163122,
 -0.803436791734821,
 -0.012820757666824242,
 -0.991257707701088,
 -0.985376210531716,
 -0.9190783955632882,
 -0.04119810868382709,
 -0.9911850690422659,
 0.7923949740644218,
 -0.9232316691234848,
 -0.3024033046675628,
 -0.9509029494896954,
 -0.4665130473310948,
 -0.9929873555002404,
 -0.9991226618366776,
 -0.9973517499185431,
 -0.11815361605364902,
 -0.997945009886467,
 0.6254618369477527,
 -0.9984822593597766,
 -0.9960992762228199,
 -0.986158123874575,
 0.1370616355633233,
 -0.9896428813688782,
 -0.9948448902744674,
 -0.9986980602853486,
 -0.8175927837619106,
 -0.7886532772872596,
 -0.9856763030617091,
 -0.99184541349000